In [32]:
import re
import pandas as pd

from unidecode import unidecode

In [2]:
def get_senioridade(df):

    df['senioridade'] = df['senioridade'].astype(str)
    
    mapa_senioridade = {
        'Júnior': ['Júnior', 'Junior', 'Jr', 'Estágio', 'Estagiário', 'Trainee', 'trainne'],
        'Pleno':  ['Pleno', 'Plena', 'Pl', 'Jr/Pl'],
        'Sênior': ['Sênior', 'Senior', 'Sr', 'Pl/Sr', 'Tech Lead', 'Líder']
    }

    for index, row in df.iterrows():
        for senioridade, termos in mapa_senioridade.items():
            for termo in termos:
                if re.search(r'\b' + termo + r'\b', row['titulo_da_vaga'], flags=re.IGNORECASE) or \
                   re.search(r'\b' + termo + r'\b', row['descricao'], flags=re.IGNORECASE):
                        df.loc[index, 'senioridade'] = senioridade

    return df

In [3]:
benefit_map = {
    'Assistência médica':                    ['medica', 'saude', 'medico'],
    'Assistência odontológica':              ['odonto', 'dentista'],
    'Assistência psicológia':                ['psicolog', 'saude mental'],
    'Auxílio academia':                      ['academia', 'gympass', 'gym', 'total pass'],
    'Auxílio combustível':                   ['combustivel'],
    'Auxílio creche':                        ['creche'],
    'Auxílio desenvolvimento':               ['desenvolvimento'],
    'Auxílio estacionamento':                ['estacionamento'],
    'Auxílio farmácia':                      ['farmacia', 'medicamento'],
    'Auxílio fretado':                       ['fretado'],
    'Auxílio home office':                   ['auxilio home', 'custo home'],
    'Bicicletário':                          ['bicleta'],
    'Bolsa auxílio':                         ['bolsa auxilio'],
    'Café da manhã':                         ['cafe da manha'],
    'Cesta básica':                          ['cesta basica'],
    'Cesta de natal':                        ['natal'],
    'Clube de vantagens':                    ['vantagens'],
    'Consignado':                            ['consignado'],
    'Convênio com empresas parceiras':       ['convenio'],
    'Cooperativa de crédito':                ['cooperativa de credito'],
    'Day off aniversário':                   ['aniversario'],
    'Desconto em produtos':                  ['desconto em produtos'],
    'Ginástica laboral':                     ['ginastica laboral'],
    'Horário flexível':                      ['horario flexivel', 'flexibilidade'],
    'Massoterapia':                          ['massoterapia', 'massagem'],
    'Participação nos Lucros ou Resultados': ['lucros', 'plr', 'ppr'],
    'Plano de Aquisição de Ações':           ['aquisicao de acoes'],
    'Previdência privada':                   ['previdencia'],
    'Programa de remuneração variável':      ['remuneracao variavel', 'bonificac', 'premiac'],
    'Programa de treinamentos':              ['treinamento', 'capacitacao'],
    'Refeitório':                            ['refeitorio'],
    'Restaurante interno':                   ['restaurante'],
    'Sala de Jogos':                         ['jogos'],
    'Seguro de vida':                        ['vida'],
    'Vale-alimentação':                      ['alimentacao'],
    'Vale-cultura':                          ['vale cultura', 'vale-cultura'],
    'Vale-refeição':                         ['refeicao', 'ticket'],
    'Vale-transporte':                       ['transporte'],
}

In [4]:
def format_benefits_list(description: str) -> list[str] | None:
    """Extrai e padroniza os benefícios a partir da descrição da vaga.

    Args:
        description (str): Descrição da vaga.

    Returns:
        list[str] | None: Lista com os benefícios padronizados, ou None se não forem encontrados.

    Exemplos:
    ---
    >>> format_benefits_list('Oferecemos vale-alimentação e plano de saúde.')
    ['Vale-alimentação', 'Plano de saúde']
    """

    try:
        # Inicializa uma lista para armazenar os benefícios extraídos
        extracted_benefits = []

        # Converte toda a descrição para minúsculas para facilitar a correspondência
        description_lower = unidecode(description).lower()

        # Percorre o mapa de benefícios para verificar a presença de cada benefício na descrição
        for benefit, synonyms in benefit_map.items():
            # Verifica se algum sinônimo do benefício está presente na descrição
            if any(synonym in description_lower for synonym in synonyms):
                extracted_benefits.append(benefit)

        # Retorna a lista de benefícios extraídos, removendo duplicatas e mantendo a ordem original
        if extracted_benefits:
            return list(dict.fromkeys(extracted_benefits))
        else:
            return None

    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [5]:
df = pd.read_csv('../data/data_raw/vagas_glassdoor_raw.csv')

In [6]:
df.head()

,site_da_vaga,link_site,link_origem,data_publicacao,data_expiracao,data_coleta,posicao,senioridade,titulo_da_vaga,nome_empresa,cidade,estado,modalidade,contrato,regime,pcd,codigo_vaga,descricao,skills
0,Glassdoor,https://www.glassdoor.com.br/job-listing/057-p...,www.glassdoor.com.br/Vaga/index.htm/partner/jo...,2024-02-16,NaN,2024-02-23,Cientista de dados,NaN,[057] Pessoa Engenheira de Dados Plena,ilegra,NaN,NaN,trabalho remoto,NaN,NaN,NaN,1009130217500,Você é uma pessoa apaixonada pelo que faz e bu...,"['Azure', 'NoSQL', 'Git', 'English', 'SQL', 'M..."
1,Glassdoor,https://www.glassdoor.com.br/job-listing/0726-...,www.glassdoor.com.br/Vaga/index.htm/partner/jo...,2024-02-19,NaN,2024-02-23,Cientista de dados,NaN,0726 - analista de dados - sede administrativa...,Netvagas,Palotina,PR,NaN,NaN,NaN,NaN,1009134443006,Descrição:\nHabilidades necessárias:\nSuperior...,"['Statistics', 'Computer science', 'Power BI',..."
2,Glassdoor,https://www.glassdoor.com.br/job-listing/0726-...,www.glassdoor.com.br/Vaga/index.htm/partner/jo...,2024-02-09,NaN,2024-02-23,Cientista de dados,NaN,0726 - Analista de Dados - Sede Administrativa...,Sicredi,Palotina,PR,NaN,NaN,NaN,NaN,1008695489204,Somos uma instituição que está constantemente ...,"['Statistics', 'Computer science', 'Power BI',..."
3,Glassdoor,https://www.glassdoor.com.br/job-listing/3229-...,www.glassdoor.com.br/Vaga/index.htm/partner/jo...,2024-01-30,NaN,2024-02-23,Cientista de dados,NaN,3229 Analista de dados,OEmprego,Porto Alegre,RS,NaN,NaN,NaN,NaN,1009098963816,"Salário: R$ 2.200,00\nBenefícios: VT\nRamo emp...","['Statistics', 'Computer science', 'Big data',..."
4,Glassdoor,https://www.glassdoor.com.br/job-listing/67414...,www.glassdoor.com.br/Vaga/index.htm/partner/jo...,2024-02-14,NaN,2024-02-23,Cientista de dados,NaN,6741487 - PESSOA ENGENHEIRA DE DADOS SÊNIOR - OKR,Quality Digital,NaN,NaN,trabalho remoto,NaN,NaN,NaN,1009126052954,Nós somos a Quality Digital! Saiba mais sobre ...,"['Azure', 'Business intelligence', 'Data struc..."


In [7]:
df = get_senioridade(df)

df['senioridade'].replace('NaN', 'Não informado')

df['modalidade'] = df['modalidade'].replace('trabalho remoto', 'Remoto')

df['estado'] = df['estado'].replace('Distrito Federal', 'DF')

df.loc[df['modalidade'] == 'Remoto', 'cidade'] = 'Não informado'
df.loc[df['modalidade'] == 'Remoto', 'estado'] = 'Todo o Brasil'

df['regime'] = 'CLT'

df['beneficios'] = df['descricao'].apply(format_benefits_list)

/tmp/ipykernel_9246/1901582945.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Pleno' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[index, 'senioridade'] = senioridade


In [8]:
df.head()

,site_da_vaga,link_site,link_origem,data_publicacao,data_expiracao,data_coleta,posicao,senioridade,titulo_da_vaga,nome_empresa,cidade,estado,modalidade,contrato,regime,pcd,codigo_vaga,descricao,skills,beneficios
0,Glassdoor,https://www.glassdoor.com.br/job-listing/057-p...,www.glassdoor.com.br/Vaga/index.htm/partner/jo...,2024-02-16,NaN,2024-02-23,Cientista de dados,Pleno,[057] Pessoa Engenheira de Dados Plena,ilegra,Não informado,Todo o Brasil,Remoto,NaN,CLT,NaN,1009130217500,Você é uma pessoa apaixonada pelo que faz e bu...,"['Azure', 'NoSQL', 'Git', 'English', 'SQL', 'M...","[Assistência odontológica, Auxílio academia, H..."
1,Glassdoor,https://www.glassdoor.com.br/job-listing/0726-...,www.glassdoor.com.br/Vaga/index.htm/partner/jo...,2024-02-19,NaN,2024-02-23,Cientista de dados,NaN,0726 - analista de dados - sede administrativa...,Netvagas,Palotina,PR,NaN,NaN,CLT,NaN,1009134443006,Descrição:\nHabilidades necessárias:\nSuperior...,"['Statistics', 'Computer science', 'Power BI',...","[Auxílio desenvolvimento, Seguro de vida]"
2,Glassdoor,https://www.glassdoor.com.br/job-listing/0726-...,www.glassdoor.com.br/Vaga/index.htm/partner/jo...,2024-02-09,NaN,2024-02-23,Cientista de dados,NaN,0726 - Analista de Dados - Sede Administrativa...,Sicredi,Palotina,PR,NaN,NaN,CLT,NaN,1008695489204,Somos uma instituição que está constantemente ...,"['Statistics', 'Computer science', 'Power BI',...","[Assistência odontológica, Auxílio academia, A..."
3,Glassdoor,https://www.glassdoor.com.br/job-listing/3229-...,www.glassdoor.com.br/Vaga/index.htm/partner/jo...,2024-01-30,NaN,2024-02-23,Cientista de dados,NaN,3229 Analista de dados,OEmprego,Porto Alegre,RS,NaN,NaN,CLT,NaN,1009098963816,"Salário: R$ 2.200,00\nBenefícios: VT\nRamo emp...","['Statistics', 'Computer science', 'Big data',...","[Programa de treinamentos, Seguro de vida]"
4,Glassdoor,https://www.glassdoor.com.br/job-listing/67414...,www.glassdoor.com.br/Vaga/index.htm/partner/jo...,2024-02-14,NaN,2024-02-23,Cientista de dados,Sênior,6741487 - PESSOA ENGENHEIRA DE DADOS SÊNIOR - OKR,Quality Digital,Não informado,Todo o Brasil,Remoto,NaN,CLT,NaN,1009126052954,Nós somos a Quality Digital! Saiba mais sobre ...,"['Azure', 'Business intelligence', 'Data struc...","[Assistência médica, Assistência odontológica,..."


In [9]:
df['senioridade'].value_counts()

senioridade
Sênior    222
Pleno     102
Júnior     83
Name: count, dtype: int64

In [10]:
df['regime'].value_counts()

regime
CLT    749
Name: count, dtype: int64

In [91]:
def search_job_modality(description: str) -> str:
    
    modality_search = {
        'Remoto':     ['remoto', 'home office', 'remota'],
        'Híbrido':    ['hibrido', 'hibrida'],
        'Presencial': ['presencial', 'escritorio', 'local de trabalho'],
    }

    description = unidecode(description).lower()

    for modality, search_words in modality_search.items():

        check_match = any( True if word in description else False for word in search_words )

        if check_match:
            return modality
        
    return 'Não informado'

In [92]:
df['modalidade'] = df['descricao'].apply(lambda description: search_job_modality(description))

In [93]:
df['modalidade'].value_counts()

modalidade
Não informado    376
Remoto           190
Presencial       112
Híbrido           71
Name: count, dtype: int64

In [95]:
def search_job_pcd(description: str) -> str:

    description = unidecode(description).lower()

    search_words = ['pcd', 'deficiente', 'deficiencia']

    check_match = any( True if word in description else False for word in search_words )

    if check_match:
        return 'Sim'
    
    return 'Não informado'

In [96]:
df['pcd'] = df['descricao'].apply(lambda description: search_job_pcd(description))

In [97]:
df['pcd'].value_counts()

pcd
Não informado    615
Sim              134
Name: count, dtype: int64

In [98]:
skills_map = {

    'Analytics':          ['Analytics', 'Data analysis skills'],
    'APIs':               ['APIs'],
    'Banco de dados':     ['Databases', 'Relational databases'],
    'Big data':           ['Big data'],
    'Cloud':              ['Azure', 'AWS', 'Google Cloud Platform'],
    'Dataviz':            ['Data visualization'],
    'Data mining':        ['Data mining'],
    'Deep Learning':      ['Deep learning', 'TensorFlow', 'Natural language processing'],
    'Docker':             ['Docker'],
    'Estatística':        ['Statistics', 'Statistical analysis'],
    'Excel':              ['Microsoft Excel'],
    'Git':                ['Git'],
    'Java':               ['Java'],
    'Linguagem R':        ['R'],
    'Machine Learning':   ['Machine learning', 'AI'],
    'Matlab':             ['MATLAB'],
    'Modelagem de dados': ['Data modeling'],
    'MongoDB':            ['MongoDB'],
    'MySQL':              ['MySQL'],
    'NoSQL':              ['NoSQL'],
    'Oracle':             ['Oracle'],
    'Pacote Office':      ['Microsoft Office'],
    'PostgreSQL':         ['PostgreSQL'],
    'Power BI':           ['Power BI'],
    'Python':             ['Python'],
    'Scala':              ['Scala'],
    'Spark':              ['Spark'],
    'SQL':                ['SQL'],
    'Tableau':            ['Tableau'],
    
}

In [107]:
def format_skills_list(skills_list: str):
    skills_list = re.findall(r'\'(.*?)\'', skills_list)

    formatted_skills = []

    for skill_name, match_words in skills_map.items():
        check_match = any( True if word in skills_list else False for word in match_words )

        if check_match:
            formatted_skills.append(skill_name)

    return formatted_skills

In [108]:
df['skills'] = df['skills'].apply(lambda skills_list: format_skills_list(skills_list))

In [119]:
df.rename(columns={'titulo_da_vaga': 'titulo_vaga'}, inplace=True)

columns_selected = [
            'site_da_vaga',
            'link_site',
            'link_origem',
            'data_publicacao',
            'data_expiracao',
            'data_coleta',
            'posicao',
            'titulo_vaga',
            'senioridade',
            'cidade',
            'estado',
            'modalidade',
            'nome_empresa',
            'contrato',
            'regime',
            'pcd',
            'beneficios',
            'skills',
            'codigo_vaga',
            'descricao'
]

df = df[columns_selected]

In [123]:
df.to_excel('../data/data_clean/vagas_glassdoor_clean.xlsx', index=False)

In [121]:
df.head()

,site_da_vaga,link_site,link_origem,data_publicacao,data_expiracao,data_coleta,posicao,titulo_vaga,senioridade,cidade,estado,modalidade,nome_empresa,contrato,regime,pcd,beneficios,skills,codigo_vaga,descricao
0,Glassdoor,https://www.glassdoor.com.br/job-listing/057-p...,www.glassdoor.com.br/Vaga/index.htm/partner/jo...,2024-02-16,NaN,2024-02-23,Cientista de dados,[057] Pessoa Engenheira de Dados Plena,Pleno,Não informado,Todo o Brasil,Remoto,ilegra,NaN,CLT,Não informado,"[Assistência odontológica, Auxílio academia, H...","[Cloud, Git, Machine Learning, NoSQL, Python, ...",1009130217500,Você é uma pessoa apaixonada pelo que faz e bu...
1,Glassdoor,https://www.glassdoor.com.br/job-listing/0726-...,www.glassdoor.com.br/Vaga/index.htm/partner/jo...,2024-02-19,NaN,2024-02-23,Cientista de dados,0726 - analista de dados - sede administrativa...,NaN,Palotina,PR,Não informado,Netvagas,NaN,CLT,Não informado,"[Auxílio desenvolvimento, Seguro de vida]","[Banco de dados, Estatística, Power BI, Python...",1009134443006,Descrição:\nHabilidades necessárias:\nSuperior...
2,Glassdoor,https://www.glassdoor.com.br/job-listing/0726-...,www.glassdoor.com.br/Vaga/index.htm/partner/jo...,2024-02-09,NaN,2024-02-23,Cientista de dados,0726 - Analista de Dados - Sede Administrativa...,NaN,Palotina,PR,Presencial,Sicredi,NaN,CLT,Não informado,"[Assistência odontológica, Auxílio academia, A...","[Banco de dados, Estatística, Power BI, Python...",1008695489204,Somos uma instituição que está constantemente ...
3,Glassdoor,https://www.glassdoor.com.br/job-listing/3229-...,www.glassdoor.com.br/Vaga/index.htm/partner/jo...,2024-01-30,NaN,2024-02-23,Cientista de dados,3229 Analista de dados,NaN,Porto Alegre,RS,Não informado,OEmprego,NaN,CLT,Não informado,"[Programa de treinamentos, Seguro de vida]","[Banco de dados, Big data, Estatística, SQL]",1009098963816,"Salário: R$ 2.200,00\nBenefícios: VT\nRamo emp..."
4,Glassdoor,https://www.glassdoor.com.br/job-listing/67414...,www.glassdoor.com.br/Vaga/index.htm/partner/jo...,2024-02-14,NaN,2024-02-23,Cientista de dados,6741487 - PESSOA ENGENHEIRA DE DADOS SÊNIOR - OKR,Sênior,Não informado,Todo o Brasil,Não informado,Quality Digital,NaN,CLT,Sim,"[Assistência médica, Assistência odontológica,...",[Cloud],1009126052954,Nós somos a Quality Digital! Saiba mais sobre ...


# Teste

In [ ]:
df_glassdoor = pd.read_csv('../data/data_clean/vagas_glassdoor_clean.csv')
df_glassdoor

In [ ]:
glassdoor_skills = df_glassdoor['skills']
glassdoor_skills

In [ ]:
from collections import Counter

def get_unique_skills(skills_column: pd.Series):
    
    #unique_skills = set()
    skills_counter = Counter()

    for skills in skills_column:
        skills_list = re.findall(r'\'(.*?)\'', skills)

        #unique_skills.update(skills_list)

        skills_counter.update(skills_list)

    return skills_counter

In [ ]:
get_unique_skills(glassdoor_skills)

In [ ]:
{
    'AI',
    'Analytics',
    'APIs',
    'AWS',
    'Azure',
    'Big data',
    'Business intelligence',
    'Cleaning',
    'Communication skills',
    'Computer science',
    'Continuous improvement',
    'Data analysis skills',
    'Data collection',
    'Data mining',
    'Data modeling',
    'Data science',
    'Data structures',
    'Data visualization',
    'Databases',
    'Deep learning',
    'Docker',
    'Google Cloud Platform',
    'Git',
    'Java',
    'Leadership',
    'Machine learning',
    'Management',
    'Management reporting',
    'Microsoft Excel',
    'Microsoft Office',
    'MongoDB',
    'MySQL',
    'Natural language processing',
    'NoSQL',
    'Oracle',
    'PostgreSQL',
    'Power BI',
    'Python',
    'R',
    'Relational databases',
    'Scala',
    'Software development',
    'Spark',
    'SQL',
    'Statistics',
    'Statistical analysis',
    'Scrum',
    'Tableau',
    'TensorFlow',
}

In [ ]:
glassdoor_skills = {

    'Analytics':          ['Analytics', 'Data analysis skills'],
    'APIs':               ['APIs'],
    'Banco de dados':     ['Databases', 'Relational databases'],
    'Big data':           ['Big data'],
    'Cloud':              ['Azure', 'AWS', 'Google Cloud Platform'],
    'Dataviz':            ['Data visualization'],
    'Data mining':        ['Data mining'],
    'Deep Learning':      ['Deep learning', 'TensorFlow', 'Natural language processing'],
    'Docker':             ['Docker'],
    'Estatística':        ['Statistics', 'Statistical analysis'],
    'Excel':              ['Microsoft Excel'],
    'Git':                ['Git'],
    'Java':               ['Java'],
    'Linguagem R':        ['R'],
    'Machine Learning':   ['Machine learning', 'AI'],
    'Matlab':             ['MATLAB'],
    'Modelagem de dados': ['Data modeling'],
    'MongoDB':            ['MongoDB'],
    'MySQL':              ['MySQL'],
    'NoSQL':              ['NoSQL'],
    'Oracle':             ['Oracle'],
    'Pacote Office':      ['Microsoft Office'],
    'PostgreSQL':         ['PostgreSQL'],
    'Power BI':           ['Power BI'],
    'Python':             ['Python'],
    'Scala':              ['Scala'],
    'Spark':              ['Spark'],
    'SQL':                ['SQL'],
    'Tableau':            ['Tableau'],

    # 'Business intelligence',
    # 'Computer science',
    # 'Continuous improvement',
    # 'Data collection',
    # 'Data structures',
    
    
    # 'Leadership',
    # 'Management',
    # 'Management reporting',
    
}

In [8]:
df.to_csv('../data/data_clean/vagas_glassdoor_clean.csv', index=False)